In [9]:
import pandas as pd
import numpy as np
import collections
import torch
import re

In [81]:
def preprocess_text(text):
    if type(text) == float:
        print(text)
    text = text.lower()
    text = re.sub(r"([.,!?])", r" \1 ", text)    # Add one whitespace before and after the characters listed.
    text = re.sub(r"[^a-zA-Z.,!?]+", r" ", text) # Remove characters different to the ones listed.
    
    return text

In [59]:
train_reviews = pd.read_csv("../data/YELP/raw_train.csv", header=None, names=["rating","review"])
train_reviews = train_reviews[~pd.isnull(train_reviews)]

test_reviews = pd.read_csv("../data/YELP/raw_test.csv", header=None, names=["rating","review"])
test_reviews = test_reviews[~pd.isnull(test_reviews)]

In [47]:
train_reviews.head()

,rating,review
0,1,"Unfortunately, the frustration of being Dr. Go..."
1,2,Been going to Dr. Goldberg for over 10 years. ...
2,1,I don't know what Dr. Goldberg was like before...
3,1,I'm writing this review to give you a heads up...
4,2,All the food is great here. But the best thing...


In [60]:
train_split_by_rating = collections.defaultdict(list)

for _, row in train_reviews.iterrows():
    train_split_by_rating[row.rating].append(row.to_dict())

In [68]:
final_list = []
np.random.seed(42)

for _, item_list in train_split_by_rating.items():
    np.random.shuffle(item_list)
    n_total = len(item_list)
    n_train = int(0.8*n_total)
    n_val = int(0.1*n_total)
    
    for item in item_list[:n_train]:
        item["split"] = "train"
        
    for item in item_list[n_train:]:
        item["split"] = "val"
    
    final_list.extend(item_list)

In [70]:
for _, row in test_reviews.iterrows():
    new_row = row.to_dict()
    new_row["split"] = "test"
    final_list.append(new_row)

In [71]:
preprocess_reviews = pd.DataFrame(final_list)
#preprocess_reviews.split.value_counts()
preprocess_reviews.review = preprocess_reviews.review.apply(preprocess_text)
preprocess_reviews.rating = preprocess_reviews.rating.apply({1:"negative", 2:"positive"}.get)

test     38000
train       15
val          5
Name: split, dtype: int64